In [3]:
import numpy as np
import control.matlab as m
import matplotlib.pyplot as plt

from pathlib import Path

PATH_IMAGES = Path('images')


# Dados de Entrada

In [4]:
fpass   = 2772    #[Hz] 
fstop   = 2272    #[Hz]
apass   = -3      #[dB]
astop   = -18     #[dB]
omegarH = fpass/fstop
omega0  = 2*np.pi*fpass

# Funções úteis

In [5]:
def thetam (n:int)->np.ndarray:
    aux     = 0 if n%2 == 0 else 1
    mfinal  = (n-aux)/2 - 1
    m       = np.arange(0, mfinal+1)
    
    return np.pi*(2*m + n + 1)/(2*n)

def Hb(n:int, R:float, w0:float = 1)-> m.TransferFunction:
        sigma_m = R*np.cos(thetam(n))
        omega_m = R*np.sin(thetam(n))
        exp     = 0 if n%2 == 0 else 1
        
        S   = m.tf('s')
        B1 = -2*sigma_m
        B2 = sigma_m**2 + omega_m**2

        H = R**(exp)*np.prod(B2)/((S+R)**exp)
        for i in range(len(sigma_m)):
            H*=1/(S**2 + B1[i]*S + B2[i])
    
        print(f'Hb = {H}')
        return H
        

epsf = lambda a: np.sqrt(10**(-0.1*a) -1)


# 1. Filtro de Butterworth

### 1.1 Filtro passa-baixa normalizado

In [6]:


# fator de ajuste
eps = epsf(apass)

# ordem do filtro
nb  = np.log10(epsf(astop)/epsf(apass))/(2*np.log10(omegarH))
n   =  round(nb)+1 if not nb%1 == 0 else nb

# alocação dos polos
R = eps**(-1/n)
sigma_m = R*np.cos(thetam(n))
omega_m = R*np.sin(thetam(n))

print(f'epsilon\t= {round(eps,3)}')
print(f'nb\t= {round(nb,3)} ({n}th order)')
print(f'sigma_m\t= {sigma_m}')
print(f'omega_m\t= {omega_m} [rad/s]')
if n%2 == 0:
    print(f'sigma_r\t= {-R}')

# função de transferência
H = Hb(n,R)


epsilon	= 0.998
nb	= 5.195 (6th order)
sigma_m	= [-0.25892149 -0.70738667 -0.96630816]
omega_m	= [0.96630816 0.70738667 0.25892149] [rad/s]
sigma_r	= -1.0003958236036716
Hb = 
                               1.002
--------------------------------------------------------------------
s^6 + 3.865 s^5 + 7.47 s^4 + 9.152 s^3 + 7.476 s^2 + 3.871 s + 1.002



In [9]:
s = m.tf('s')
print(H(s/2))

TypeError: must be real number, not TransferFunction